In [1]:
import os
import sys

In [2]:
%pwd

'e:\\6_learning\\Kidney-Disease-Classification-DLproject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\6_learning\\Kidney-Disease-Classification-DLproject'

In [5]:
# entity >> return type of data ingestion fuction

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# Go 2 levels up from research/ to project root
ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

In [ ]:
!pip install -e .

^C


Obtaining file:///E:/6_learning/Kidney-Disease-Classification-DLproject
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: src
    Found existing installation: src 0.0.0
    Uninstalling src-0.0.0:
      Successfully uninstalled src-0.0.0
  Running setup.py develop for src


  DEPRECATION: Legacy editable install of src==0.0.0 from file:///E:/6_learning/Kidney-Disease-Classification-DLproject (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [8]:
from src.constants import *
from src.utils.common import create_directories, read_yaml

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import zipfile
import gdown
from src import logging
from src.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logging.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[5]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix+file_id,zip_download_dir,quiet=False,verify="C:\\path\\to\\company-ca-cert.pem")

            logging.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[ 2025-09-03 08:28:43,543 ] 31 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2025-09-03 08:28:43,551 ] 31 root - INFO - yaml file: params.yaml loaded successfully
[ 2025-09-03 08:28:43,554 ] 51 root - INFO - created directory at: artifacts
[ 2025-09-03 08:28:43,560 ] 51 root - INFO - created directory at: artifacts/data_ingestion
[ 2025-09-03 08:28:43,564 ] 15 root - INFO - Downloading data from https://drive.google.com/file/d/1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl/view into file artifacts/data_ingestion/data.zip


OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\path\to\company-ca-cert.pem

In [26]:
import os
os.chdir(r"E:\6_learning\Kidney-Disease-Classification-DLproject")
print(os.getcwd())  # should now show the project root

E:\6_learning\Kidney-Disease-Classification-DLproject


In [14]:
import os
import sys
import zipfile
import gdown
import certifi
import logging
from pathlib import Path
from dataclasses import dataclass
from src.utils.common import create_directories, read_yaml
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

# ---------------------------
# Fix ROOT handling
# ---------------------------
ROOT = Path.cwd().parent   # always project root
sys.path.append(str(ROOT))


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """Fetch data from Google Drive link."""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file

            logging.info(f"Downloading data from {dataset_url} into {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]  # robust ID extraction
            prefix = 'https://drive.google.com/uc?export=download&id='
            
            gdown.download(
                prefix + file_id,
                str(zip_download_dir),
                quiet=False,
                verify=certifi.where()
            )

            logging.info(f"Downloaded data into {zip_download_dir}")
            return str(zip_download_dir)

        except Exception as e:
            logging.error(f"Data download failed: {e}")
            raise e

    def extract_zip_file(self):
        """Extract zip file into target directory."""
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logging.info(f"Extracted {self.config.local_data_file} to {unzip_path}")

        except Exception as e:
            logging.error(f"Failed to extract zip: {e}")
            raise e


if __name__ == "__main__":
    try:
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
    except Exception as e:
        logging.error(f"Pipeline failed: {e}")
        raise e


[ 2025-08-29 09:59:41,325 ] 31 root - INFO - yaml file: config\config.yaml loaded successfully


[ 2025-08-29 09:59:41,338 ] 31 root - INFO - yaml file: params.yaml loaded successfully
[ 2025-08-29 09:59:41,343 ] 51 root - INFO - created directory at: artifacts
[ 2025-08-29 09:59:41,347 ] 51 root - INFO - created directory at: artifacts/data_ingestion
[ 2025-08-29 09:59:41,350 ] 60 root - INFO - Downloading data from https://drive.google.com/file/d/1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl/view into artifacts/data_ingestion/data.zip
[ 2025-08-29 09:59:42,788 ] 76 root - ERROR - Data download failed: HTTPSConnectionPool(host='doc-14-ac-docs.googleusercontent.com', port=443): Max retries exceeded with url: /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qdghnfs87e8fuk0bv8vgf2h8limfmkqj/1756441725000/01835322644805217484/*/1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl?e=download&uuid=4bac242c-bf90-4f21-93ed-4af057c9e00f (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:992)')))
[ 2025-08-29 09

SSLError: HTTPSConnectionPool(host='doc-14-ac-docs.googleusercontent.com', port=443): Max retries exceeded with url: /docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qdghnfs87e8fuk0bv8vgf2h8limfmkqj/1756441725000/01835322644805217484/*/1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl?e=download&uuid=4bac242c-bf90-4f21-93ed-4af057c9e00f (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:992)')))

In [ ]:
import certifi

In [19]:
prefix = 'https://drive.google.com/uc?export=download&id='
dataset_url = r'https://drive.google.com/file/d/1dXJvoFA95jfTKY_2sbvr4pdGDCwjpfRl/view'
file_id = dataset_url.split("/")[-2]
cert_path = r"C:\certs\corporate-ca.pem"
zip_download_dir = 'artifacts/data_ingestion/data.zip'

gdown.download(
    prefix + file_id,
    zip_download_dir,
    quiet=False,
    verify=False
)

e:\6_learning\Kidney-Disease-Classification-DLproject\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
e:\6_learning\Kidney-Disease-Classification-DLproject\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'drive.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
e:\6_learning\Kidney-Disease-Classification-DLproject\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doc-14-ac-docs.googleusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthed

'artifacts/data_ingestion/data.zip'